In [38]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET") 

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    return json_result[0]

def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=CA"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

def get_markets(token):
    url = "https://api.spotify.com/v1/markets"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result


In [39]:
token = get_token()
result = search_for_artist(token, "Of Monsters and Men")
artist_id = result["id"]
songs = get_songs_by_artist(token, artist_id)
for idx, song in enumerate(songs):
    print(f"{idx + 1}. {song['name']}")
markets = get_markets(token)

1. Little Talks
2. Dirty Paws
3. Mountain Sound
4. King And Lionheart
5. Crystals
6. Love Love Love
7. Alligator
8. Wolves Without Teeth
9. Human
10. Empire


### PLAN
**Problem:** Given a song, recommend 5 new songs.<br><br>
**Solution:**
1. Build and train the model
    1. Use the Spotify Web API's *Get Tracks' Audio Features* to get the features of tracks.
    2. Perform exploratory data analysis to find general patterns, missing values, identify outliers, etc.
    3. Perform feature engineering to remove irrelevant data, combine data, and mutate data.
    4. Use Spotify Web API's *Get Recommendations* as labeled data.
    5. Train the model.<br><br>
2. Test the model
    1. Input a song.
    2. Use the Spotify Web API to get the track's audio features.
    2. Check the proximity of model's recommended songs vs Spotify's recommended songs.<br><br>
3. Use the model
    1. Input a song.
    2. Enjoy the 5 recommended songs :)